In [11]:
# Display information about the environment
VERSION, Base.Threads.nthreads(), Base.Sys.CPU_THREADS

(v"1.5.3", 8, 8)

In [12]:
using TickTock
tick()

┌ Info:  started timer at: 2020-12-13T00:00:02.788
└ @ TickTock /Users/arihuttunen/.julia/packages/TickTock/RsTHR/src/TickTock.jl:32


In [13]:
N = 10^8

100000000

In [14]:
using Random, Distributions, BenchmarkTools, DataFrames
ENV["COLUMNS"] = 1000; ENV["LINES"] = 15;

In [15]:
d = Normal()
df = DataFrame(:data => rand(d, N));

In [16]:
out = Vector{Union{Missing, Float64}}(undef, N)

function rollingmean2(x, m, out)
    N = length(x)
    out[1:(m-1)] .= missing
    sm = mean(@view x[1:m])
    out[m] = sm
    @inbounds @simd for i in (m+1):length(x)
        sm += (x[i] - x[i-m]) / m
        out[i] = sm
    end
    return out 
end

rollingmean2 (generic function with 1 method)

In [17]:
df[!, "sma"] = rollingmean2(df.data, 10, out)
df

,data,sma
,Float64,Float64?
1,-0.578926,missing
2,-0.657218,missing
3,-0.339101,missing
4,-1.6843,missing
5,0.335653,missing
6,-1.7517,missing
7,-0.270582,missing
8,-0.460108,missing
9,1.15648,missing


In [18]:
tock()

┌ Info:          3.046797558s: 3 seconds, 46 milliseconds
└ @ TickTock /Users/arihuttunen/.julia/packages/TickTock/RsTHR/src/TickTock.jl:39


In [19]:
@benchmark rand(d, N)

BenchmarkTools.Trial: 
  memory estimate:  762.94 MiB
  allocs estimate:  2
  --------------
  minimum time:     1.270 s (0.08% GC)
  median time:      1.383 s (0.08% GC)
  mean time:        1.373 s (1.71% GC)
  maximum time:     1.457 s (0.06% GC)
  --------------
  samples:          4
  evals/sample:     1

In [20]:
@benchmark rollingmean2(df.data, 10, out)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     120.402 ms (0.00% GC)
  median time:      139.729 ms (0.00% GC)
  mean time:        139.457 ms (0.00% GC)
  maximum time:     175.595 ms (0.00% GC)
  --------------
  samples:          36
  evals/sample:     1